### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential

### Load NSL-KDD dataset


In [2]:
url = "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz"
column_names = ["duration", "protocol_type", "service", "flag", "src_bytes",
                "dst_bytes", "land", "wrong_fragment", "urgent", "hot",
                "num_failed_logins", "logged_in", "num_compromised", "root_shell",
                "su_attempted", "num_root", "num_file_creations", "num_shells",
                "num_access_files", "num_outbound_cmds", "is_host_login",
                "is_guest_login", "count", "srv_count", "serror_rate",
                "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
                "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
                "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate",
                "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
                "dst_host_srv_diff_host_rate", "dst_host_serror_rate",
                "dst_host_srv_serror_rate", "dst_host_rerror_rate",
                "dst_host_srv_rerror_rate", "label"]
df = pd.read_csv(url, names=column_names)
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


### Drop non-numeric columns and label column

In [3]:
# Drop non-numeric columns and label column
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df_numeric = df[numeric_columns]

### Heatmap of Correlation between Features


In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(df_numeric.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap of Correlation between Features')
plt.show()

### Data Pre-processing

In [4]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(df_numeric)

# Split the data into training and test sets
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

### Define the autoencoder model

In [5]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(X_train.shape[1],activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

### Train the autoencoder

In [6]:
model.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))

Epoch 1/50
12351/12351 [==============================] - 28s 2ms/step - loss: 0.7538 - val_loss: 0.7164
Epoch 2/50
12351/12351 [==============================] - 27s 2ms/step - loss: 0.7496 - val_loss: 0.7158
Epoch 3/50
 5589/12351 [============>.................] - ETA: 16s - loss: 0.7220

KeyboardInterrupt: 

### Training and Validation Loss Over Epochs

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(model.history.history['loss'], label='Training Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


### Evaluating the model

In [ ]:
# Use the trained autoencoder to encode and decode the test data
encoded_data = model.predict(X_test)

# Evaluate the reconstruction error
mse = np.mean(np.square(X_test - encoded_data))
print("Mean Squared Error:", mse)

### Function to detect anomalies

In [ ]:
# Function to detect anomalies
def detect_anomalies(X_test, threshold):
    reconstructed = model.predict(X_test)
    reconstruction_errors = np.mean(np.power(X_test - reconstructed, 2), axis=1)
    anomaly_indices = np.where(reconstruction_errors > threshold)[0]
    return anomaly_indices, reconstruction_errors

In [ ]:
threshold = 0.3  
anomaly_indices, reconstruction_errors = detect_anomalies(X_test, threshold)
print("Anomaly indices:", anomaly_indices)
print("Reconstruction errors:", reconstruction_errors)

### Histogram of Reconstruction Errors


In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(reconstruction_errors, bins=30)
plt.title('Histogram of Reconstruction Errors')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.show()

### Scatter Plot of Reconstruction Errors vs. Data Indices


In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(range(len(reconstruction_errors)), reconstruction_errors, alpha=0.5)
plt.title('Scatter Plot of Reconstruction Errors vs. Data Indices')
plt.xlabel('Data Indices')
plt.ylabel('Reconstruction Error')
plt.show()

### Threshold Line on Reconstruction Error Scatter Plot

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(range(len(reconstruction_errors)), reconstruction_errors, alpha=0.5)
plt.axhline(y=threshold, color='r', linestyle='--', label='Threshold')
plt.title('Reconstruction Errors with Threshold Line')
plt.xlabel('Data Indices')
plt.ylabel('Reconstruction Error')
plt.legend()
plt.show()

### Bar Plot of Anomaly Counts per Feature

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x=anomaly_indices, data=df.columns)
plt.title('Anomaly Counts per Feature')
plt.xlabel('Feature')
plt.ylabel('Anomaly Count')
plt.xticks(rotation=90)
plt.show()

### Box Plot of Reconstruction Errors by Class

In [ ]:

plt.figure(figsize=(10, 5))
sns.boxplot(x=df['label'], y=reconstruction_errors)
plt.title('Box Plot of Reconstruction Errors by Class')
plt.xlabel('Class')
plt.ylabel('Reconstruction Error')
plt.show()